In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [4]:
raw=pd.read_csv(r"C:\Users\user\Documents\Intership\Task 1\netflix_titles.csv")
df=raw.copy()

In [6]:
print("shape:",df.shape)
display(df.head(3))

shape: (8807, 12)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...


In [7]:
display(df.info(3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


None

In [8]:
display(df.isnull().sum())

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

## Normalize the Column Names

In [9]:
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
df.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."


## Remove Exact Duplicates

In [10]:
orig_row=df.shape[0]
dups=df.duplicated().sum()
print("Exact Duplicates:",dups)
df.drop_duplicates(inplace=True)
print("Rpws after removing duplicates:",df.shape[0])

Exact Duplicates: 0
Rpws after removing duplicates: 8807


## Tidy date Column

In [15]:
df['date_added']=pd.to_datetime(df['date_added'], errors='coerce')

df['release_year']=pd.to_numeric(df['release_year'], errors='coerce').astype('Int64')
df['date_added']=df['date_added'].fillna(pd.to_datetime(df['release_year'].astype(str)+'-01-01', errors='coerce'))


df['added_year']=df['date_added'].dt.year
df['added_month']=df['date_added'].dt.month


## Standardize categorical text (type, rating, listed_in)

In [17]:
df['type']=df['type'].str.strip().str.title()
df['type']=df['type'].replace({'Tv Show':'TV Show'})

df['rating'] = df['rating'].str.strip().replace('', np.nan)
df['rating'] = df['rating'].fillna('Not Rated')


df['listed_in'] = df['listed_in'].astype(str).apply(lambda x: [g.strip() for g in x.split(',')])
df['primary_genre'] = df['listed_in'].apply(lambda x: x[0] if x else None)

## Parse & standardize duration (minutes vs seasons)

In [18]:
def parse_duration(x):
    if pd.isna(x):
        return pd.Series({'duration_int': pd.NA, 'duration_type': pd.NA})
    x = str(x).strip()
    m = re.search(r'(\d+)', x)
    if not m:
        return pd.Series({'duration_int': pd.NA, 'duration_type': pd.NA})
    val = int(m.group(1))
    if 'season' in x.lower():
        return pd.Series({'duration_int': val, 'duration_type': 'seasons'})
    else:
        return pd.Series({'duration_int': val, 'duration_type': 'minutes'})

df[['duration_int', 'duration_type']] = df['duration'].apply(parse_duration)
df['duration_int'] = df['duration_int'].astype('Int64')

## Handle country, director, cast missingness & structure

In [19]:
df['country'] = df['country'].astype(str).replace('nan','')
df['country_list'] = df['country'].apply(lambda x: [c.strip() for c in x.split(',')] if x and x.strip() else [])
df['primary_country'] = df['country_list'].apply(lambda lst: lst[0] if lst else 'Unknown')

df['director'] = df['director'].fillna('Unknown')

df['cast'] = df['cast'].fillna('')
df['cast_list'] = df['cast'].apply(lambda x: [c.strip() for c in x.split(',')] if x else [])
df['num_cast'] = df['cast_list'].apply(len)

## Clean text fields (title, description)

In [21]:
df['title'] = df['title'].str.strip()

df['description'] = df['description'].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()

## Convert datatypes & final dtype checks

In [23]:
df['release_year'] = pd.to_numeric(df['release_year'], errors='coerce').astype('Int64')

display(df.dtypes)
display(df.isnull().sum())

show_id                    object
type                       object
title                      object
director                   object
cast                       object
country                    object
date_added         datetime64[ns]
release_year                Int64
rating                     object
duration                   object
listed_in                  object
description                object
added_year                  int32
added_month                 int32
primary_genre              object
duration_int                Int64
duration_type              object
country_list               object
primary_country            object
cast_list                  object
num_cast                    int64
dtype: object

show_id            0
type               0
title              0
director           0
cast               0
country            0
date_added         0
release_year       0
rating             0
duration           3
listed_in          0
description        0
added_year         0
added_month        0
primary_genre      0
duration_int       3
duration_type      3
country_list       0
primary_country    0
cast_list          0
num_cast           0
dtype: int64

## Remove duplicate titles (fuzzy / subset duplicates)

In [25]:
dup_subset = df.duplicated(subset=['title', 'type', 'release_year']).sum()
print("duplicates by title/type/year:", dup_subset)
df = df.drop_duplicates(subset=['title', 'type', 'release_year'], keep='first')


duplicates by title/type/year: 0


## Re-check Dataset

In [27]:
final_rows = df.shape[0]
print("original rows:", orig_row, " final rows:", final_rows)
display(df.isnull().sum())

original rows: 8807  final rows: 8806


show_id            0
type               0
title              0
director           0
cast               0
country            0
date_added         0
release_year       0
rating             0
duration           3
listed_in          0
description        0
added_year         0
added_month        0
primary_genre      0
duration_int       3
duration_type      3
country_list       0
primary_country    0
cast_list          0
num_cast           0
dtype: int64

## Export cleaned CSV

In [36]:
df.to_csv("Data/Cleaned_Netflix_dataset.csv", index=False)

## Short Summary of Changes

1. Rows before / after, duplicates removed (counts).

2. Columns that had significant missing values and how you handled them:
    1. director: 2634 missing → filled with "Unknown"
    2. country: 831 missing → set primary_country = 'Unknown' and saved full country_list
    3. date_added: 10 missing → filled with Jan 1 of release_year
    4. rating: 4 missing → filled with "Not Rated"

3. New columns added: (duration_int, duration_type, primary_genre, primary_country, num_cast)

4. Any rows removed by rule (e.g., exact duplicates or subset duplicates).